In [1]:
from simtk import unit, openmm
from simtk.openmm import app
from alchemy import AbsoluteAlchemicalFactory, AlchemicalState

import blues.utils as utils
import blues.ncmc as ncmc
import blues.ncmc_switching as ncmc_switching
from blues.smartdart import SmartDarting

import sys
import numpy as np
import mdtraj as md
from mdtraj.reporters import HDF5Reporter
from datetime import datetime
from optparse import OptionParser

In [90]:
# Define some constants
temperature = 300.0*unit.kelvin
friction = 1/unit.picosecond
dt = 0.002*unit.picoseconds
# set nc attributes
numIter = 10
nstepsNC = 10
nstepsMD = 50
#Defines ncmc move eqns for lambda peturbation of sterics/electrostatics
functions = { 'lambda_sterics' : 'step(0.199999-lambda) + step(lambda-0.2)*step(0.8-lambda)*abs(lambda-0.5)*1/0.3 + step(lambda-0.800001)',
            'lambda_electrostatics' : 'step(0.2-lambda)- 1/0.2*lambda*step(0.2-lambda) + 1/0.2*(lambda-0.8)*step(lambda-0.8)' }

In [122]:
# Obtain topologies/positions
coord_file = 'eqToluene.inpcrd'
top_file =   'eqToluene.prmtop'
prmtop = app.AmberPrmtopFile(top_file)
inpcrd = app.AmberInpcrdFile(coord_file)
# Generate OpenMM System
system = prmtop.createSystem(nonbondedMethod=app.PME,
                            nonbondedCutoff=1*unit.nanometer,
                            constraints=app.HBonds)

In [123]:
# Initailize MD Simulation
mdIntegrator = openmm.LangevinIntegrator(temperature, friction, dt)
mdSim = app.Simulation(prmtop.topology, system, mdIntegrator)
mdSim.context.setPositions(inpcrd.positions)
mdSim.context.setVelocitiesToTemperature(temperature)
mdSim.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
# Add reporters for MD simulation
mdSim.reporters.append(app.dcdreporter.DCDReporter('traj.dcd', nstepsMD))
#mdSim.reporters.append(HDF5Reporter('traj.h5', nstepsMD))

In [150]:
class SimulateNCMC(object):
    def __init__(self, md_simulation, resname, 
                 platform=None, platformProperties=None, state=None):
        
        #Set reference variables from MD
        self.topology = md_simulation.topology
        self.system = md_simulation.system
        self.integrator = md_simulation.integrator
        self.resname = resname
        self.box_vectors = self.system.getDefaultPeriodicBoxVectors()
        #self.context = md_simulation.context
        
        # Set physical parameters
        self.temperature = 300.0*unit.kelvin
        self.friction = 1/unit.picosecond
        self.dt = 0.002*unit.picoseconds
        kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
        self.beta = 1.0 / (kB * temperature)
        
        self.numIter = 10
        self.nstepsNC = 10
        self.nstepsMD = 50 
        
        ## The index of the current time step
        self.currentStep = 0
        ## A list of reporters to invoke during the simulation
        self.reporters = []
        
        #Defines ncmc move eqns for lambda peturbation of sterics/electrostatics
        self.functions = {
            'lambda_sterics' : 'step(0.199999-lambda) + step(lambda-0.2)*step(0.8-lambda)*abs(lambda-0.5)*1/0.3 + step(lambda-0.800001)',
            'lambda_electrostatics' : 'step(0.2-lambda)- 1/0.2*lambda*step(0.2-lambda) + 1/0.2*(lambda-0.8)*step(lambda-0.8)' 
        }
        
        self.totalmass = 0
        self.masslist = []
        self.atomsIdx = []
        self.alchemicalSystem = None
        self.alchemicalSim = None
        self.ncContext = None
        self.ncPositions = None
        self.positions = None
        
        self.acceptance = 0
        self.nc_integrator = None
        
        self._storage = None
        self.com = None
        self.rotation = None
        
       

    def setAtomsIdx(self):
        idx = []
        for atom in self.topology.atoms():
            if str(self.resname) in atom.residue.name:
                idx.append(atom.index)
        self.atomsIdx = idx
        print('Ligand atoms:', self.atomsIdx)
        return self.atomsIdx
    
    def generateAlchemicalSimulation(self):
        #Initialize Alchemical Simulation
        # performs alchemical corrections
        # Reporter for NCMC moves
        alchemicalIntegrator = openmm.LangevinIntegrator(self.temperature, 
                                                      self.friction,
                                                      self.dt)
        alchemicalSim = app.Simulation(self.topology, self.system, alchemicalIntegrator)
        alchemicalSim.context.setPeriodicBoxVectors(*self.box_vectors)
        self.alchemicalSim = alchemicalSim
        print('Generated Alchemical Simulation:', type(alchemicalSim))
        return self.alchemicalSim
    
    def generateAlchemicalSystem(self):        
        # Generate Alchemical System
        factory = AbsoluteAlchemicalFactory(self.system, ligand_atoms=self.atomsIdx,
                                            annihilate_sterics=True,
                                            annihilate_electrostatics=True)
        alchemicalSystem = factory.createPerturbedSystem()
        self.alchemicalSystem = alchemicalSystem
        print('Generated Alchemical System:', type(alchemicalSystem))
        return self.alchemicalSystem
    
    def generateNCContext(self):
        ncIntegrator = ncmc_switching.NCMCVVAlchemicalIntegrator(self.temperature, 
                                                                 self.alchemicalSystem,
                                                                 self.functions,
                                                                 nsteps=self.nstepsNC)
        ncContext = openmm.Context(self.alchemicalSystem, ncIntegrator)
        ncContext.setPeriodicBoxVectors(*self.box_vectors)
        self.ncContext = ncContext
        print('Generated NCMC context:', type(ncContext))
        return self.ncContext
    
    def getMasses(self):
        masses = unit.Quantity(np.zeros([len(self.atomsIdx),1],np.float32), unit.dalton)
        #sytem = context.getSystem()
        for ele, idx in enumerate(self.atomsIdx):
            masses[ele] = self.system.getParticleMass(idx)
        self.masses = masses
        self.totalmass = self.getTotalMass(masses)
        return self.masses
    
    def getTotalMass(self, masses):
        self.totalmass = masses.sum()
        return self.totalmass
    
    def getCOM(self):
        #positions = self.getPositions(context, atomsIdx)
        com =  (masses / totalmass * positions).sum(0)
        self.com = com
        return self.com

    def getPositions(self, context, index=[] ):
        state = context.getState(getPositions=True)
        pos_array = state.getPositions(asNumpy=True)
        coordinates = pos_array / unit.nanometers
        if not index:
            index = range(len(coordinates))
        else:
            index = self.atomsIdx
        positions = unit.Quantity(np.zeros( [len(index),3], np.float32), unit.nanometers)
        for e, i in enumerate(index):
            positions[e,:] = unit.Quantity(coordinates[i], unit.nanometers)
        return positions

    def generateRotation(self):
        comPos = self.getCOM(context, atomsIdx)
        alchPos = self.getPositions(context, atomsIdx)
        # Remove COM from alchemical positions to perform rotation
        tempPos = alchPos - comPos
        # Dot alchPostions with rotation matrix,
        rotPos = unit.Quantity( np.dot(tempPos, rand_rotation_matrix()), unit.nanometers)
        rotation = rotPos + comPos
        self.rotation = rotation
        return self.rotation
    
    def rand_rotation_matrix():
        rand_quat = md.utils.uniform_quaternion()
        matrix_out = md.utils.rotation_matrix_from_quaternion(rand_quat)
        return matrix_out
        
    def updatePositions(self, context, index=[], positions):
        #Store reference positions
        self.positions = self.getPositions(self.context)
        for ele, idx in enumerate(atomsIdx):
            positions[idx] = newpositions[ele]
        self.positions = positions
        self.context = context.setPositions(positions)
        return self.positions
    
    def _prepNCMC(self):
        #First Generate MD Simulation object
        self.setAtomsIdx(self.resname, self.topology)
        #Generate Alchemical System/contexts
        self.generateAlchemicalSystem()
        self.generateAlchemicalSimulation()
        #Generate NC contexts
        self.generateNCContext()
        
    def _simulation(self): 
        ### Set Initial Conditions
        #Set NC positions to reference from MD
        mdState = md_simulation.context.getState(getPositions=True,
                                                  getVelocities=True,
                                                  getEnergy=True,
                                                  getParameters=True,
                                                  enforcePeriodicBox=True)
        ncState = ncContext.getState(getPositions=True)
        
        self.ncPositions = self.getPositions(self.ncContext)
        
        #Define our rotational move
        #Update masses for current context
        masses = self.getMasses()
        totalmass = self.getTotalMass(masses)
        #self.getMasses()
        

In [174]:
testSim = SimulateNCMC(mdSim, 'LIG')
testSim.setAtomsIdx()
testSim.generateAlchemicalSystem()
testSim.generateAlchemicalSimulation()
nc_context = testSim.generateNCContext()

Ligand atoms: [2634, 2635, 2636, 2637, 2638, 2639, 2640, 2641, 2642, 2643, 2644, 2645, 2646, 2647, 2648]
Generated Alchemical System: <class 'simtk.openmm.openmm.System'>
Generated Alchemical Simulation: <class 'simtk.openmm.app.simulation.Simulation'>
Generated NCMC context: <class 'simtk.openmm.openmm.Context'>


In [223]:
def getPositions(context, index=[]):
    pos_array = context.getState(getPositions=True).getPositions(asNumpy=True)
    coordinates = pos_array / unit.nanometers
    if not index:
        print('index is none')
        index = range(len(coordinates))
        print(index)
    else:
        index = [2634, 2635, 2636, 2637, 2638, 2639, 2640, 2641, 2642, 2643, 2644, 2645, 2646, 2647, 2648] 
        print('index is not none')
    positions = unit.Quantity(np.zeros( [len(index),3], np.float32), unit.nanometers)
    for e, i in enumerate(index):
        positions[e,:] = unit.Quantity(coordinates[i], unit.nanometers)
    print(positions)
    return positions

In [224]:

nc_pos = getPositions(nc_context)

index is none
range(0, 22340)
[[ 3.34299994  1.12800002  4.44899988]
 [ 3.28600001  1.18700004  4.50899982]
 [ 3.29699993  1.03799999  4.44399977]
 ..., 
 [ 1.949       3.5150001   0.73299998]
 [ 1.97300005  3.56299996  0.81199998]
 [ 1.87800002  3.45700002  0.75999999]] nm


In [215]:
range(len(pos_array))

range(0, 22340)

In [153]:
nc_pos

Quantity(value=array([], shape=(0, 3), dtype=float32), unit=nanometer)

In [73]:
# Generate NC Integrator/Contexts
ncIntegrator = ncmc_switching.NCMCVVAlchemicalIntegrator(temperature, alchemySystem, functions,
                                          nsteps=nstepsNC)
nc_context = openmm.Context(alchemySystem, ncIntegrator)
nc_context.setPeriodicBoxVectors(*inpcrd.boxVectors)

In [75]:
# Initialize BLUES engineb
blues_run = SimNCMC(temperature, ligand_atoms)

In [76]:
rot_step = (nstepsNC/2) -1
nc_move = [[blues_run.rotationalMove, [rot_step]]]
blues_run.getParticleMass(system, ligand_atoms)
blues_run.runSim(mdSim, nc_context, ncIntegrator,
                    alchemySim, movekey=nc_move,
                    niter=numIter, nstepsNC=nstepsNC, nstepsMD=nstepsMD,
                    alchemical_correction=True)

performing ncmc step
accCounter = 0
doing the move
before step 0.4
[[ 2.44499752  2.3829845   2.83696856]
 [ 2.31063967  2.3696759   2.88333503]
 [ 2.54539233  2.29625125  2.88578685]
 [ 2.2796032   2.28091881  2.97956349]
 [ 2.50563998  2.19912418  2.9781453 ]
 [ 2.374742    2.18761226  3.02316966]
 [ 2.34198516  2.08976972  3.12664346]
 [ 2.46779674  2.46165307  2.76560714]
 [ 2.23039163  2.4344837   2.84741568]
 [ 2.64736206  2.31609139  2.85482651]
 [ 2.17994698  2.26818445  3.02302005]
 [ 2.58200596  2.12790552  3.01186682]
 [ 2.28190674  2.13754955  3.20486539]
 [ 2.28309936  2.00833523  3.08190101]
 [ 2.42734837  2.03475159  3.16501177]] rot_output
[ 2.237  2.168  2.977] [ 2.44499752  2.3829845   2.83696856]
[ 2.26   2.293  3.042] [ 2.31063967  2.3696759   2.88333503]
[ 2.34   2.108  2.901] [ 2.54539233  2.29625125  2.88578685]
[ 2.376  2.359  3.025] [ 2.2796032   2.28091881  2.97956349]
[ 2.462  2.176  2.895] [ 2.50563998  2.19912418  2.9781453 ]
[ 2.483  2.298  2.958] [ 2.3747

Exception: Particle coordinate is nan

In [ ]:
# Pack into Simulation object for easy data retrieval 
ncSim = app.Simulation(prmtop.topology, alchemySystem, ncIntegrator)
ncSim.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

In [ ]:
# Simulation Object contains:
#  Topology/System/Integrator/currentStep/Reporters/Context
#   Context contains: Positions/Velocities/Box/State

alchemyPositions = []
for idx in ligand_atoms:
    pos = unit.Quantity(coordinates[idx], unit.angstroms)
    alchemyPositions.append(pos)
alchemyPositions = unit.Quantity(alchemyPositions)

In [ ]:
dummy = NCMCModeller()
masses = dummy.getMasses(mdSim.context, ligand_atoms)
totalmass = dummy.getTotalMass(masses)
positions = dummy.getPositions(mdSim.context, ligand_atoms)
com = dummy.getCOM(mdSim.context, ligand_atoms)
positions

In [ ]:
rotation = dummy.generateRotation(mdSim.context, ligand_atoms)
rotation

In [ ]:
dummy.setPositions(ncSim.context, ligand_atoms, rotation)

In [ ]:
dummy.getCOM(mdSim.context, ligand_atoms)


In [ ]:
# Initialize BLUES engine
blues_run = ncmc.SimNCMC(temperature=temperature, residueList=lig_atoms)

In [ ]:
dummy = AlchemicalDummy()
dummy.getMasses(system, lig_atoms)


In [ ]:
#Define NC move
rot_step = (nstepsNC/2) -1
nc_move = [[blues_run.rotationalMove, [rot_step]]]

In [ ]:
# actually run
blues_run.get_particle_masses(system, residueList=lig_atoms)
blues_run.runSim(md_sim, nc_context, nc_integrator,
                alch_sim, movekey=nc_move,
                niter=numIter, nstepsNC=nstepsNC, nstepsMD=nstepsMD,
                alchemical_correction=True)

In [ ]:
class Simulate(object):
    def __init__(self, topology, system, integrator, 
                 platform=None, platformProperties=None, state=None):
        self.topology = topology
        ## The System being simulated
        if isinstance(system, string_types):
            with open(system, 'r') as f:
                self.system = openmm.XmlSerializer.deserialize(f.read())
        else:
            self.system = system
        ## The Integrator used to advance the simulation
        if isinstance(integrator, string_types):
            with open(integrator, 'r') as f:
                self.integrator = openmm.XmlSerializer.deserialize(f.read())
        else:
            self.integrator = integrator
        ## The index of the current time step
        self.currentStep = 0
        ## A list of reporters to invoke during the simulation
        self.reporters = []
        if platform is None:
            ## The Context containing the current state of the simulation
            self.context = mm.Context(self.system, self.integrator)
        elif platformProperties is None:
            self.context = mm.Context(self.system, self.integrator, platform)
        else:
            self.context = mm.Context(self.system, self.integrator, platform, platformProperties)
        if state is not None:
            with open(state, 'r') as f:
                self.context.setState(mm.XmlSerializer.deserialize(f.read()))
        ## Determines whether or not we are using PBC. Try from the System first,
        ## fall back to Topology if that doesn't work
        try:
            self._usesPBC = self.system.usesPeriodicBoundaryConditions()
        except Exception: # OpenMM just raises Exception if it's not implemented everywhere
            self._usesPBC = topology.getUnitCellDimensions() is not None
            
        self.totalmass = 0
        self.masses = []
        self.idx = []
        self.positions = None
        self.com = None
        self.rotation = None
        self.context = None
        self.alchemicalSystem = None
        
    def getMasses(self, context, atomsIdx):
        masses = unit.Quantity(np.zeros([len(atomsIdx),1],np.float32), unit.dalton)
        sytem = context.getSystem()
        for ele, idx in enumerate(atomsIdx):
            masses[ele] = system.getParticleMass(idx)
        self.totalmass = masses.sum()
        self.masses = masses
        return self.masses
    
    def getTotalMass(self, masses):
        self.totalmass = self.masses.sum()
        return self.totalmass
        
    def getPositions(self, context, atomsIdx):
        state = context.getState(getPositions=True)
        coordinates = state.getPositions(asNumpy=True) / unit.nanometers
        positions = unit.Quantity( np.zeros([len(atomsIdx),3],np.float32), unit.nanometers)
        for ele, idx in enumerate(atomsIdx):
            positions[ele,:] = unit.Quantity(coordinates[idx], unit.nanometers)
        self.positions = positions
        return self.positions
    
    def getCOM(self, context, atomsIdx):
        #Update masses for current context
        masses = self.getMasses(context, atomsIdx)
        totalmass = self.getTotalMass(masses)
        positions = self.getPositions(context, atomsIdx)
        com =  (masses / totalmass * positions).sum(0)
        self.com = com
        return self.com
    
    def generateRotation(self, context, atomsIdx):
        comPos = self.getCOM(context, atomsIdx)
        alchPos = self.getPositions(context, atomsIdx)
        # Remove COM from alchemical positions to perform rotation
        tempPos = alchPos - comPos
        # Dot alchPostions with rotation matrix,
        rotPos = unit.Quantity( np.dot(tempPos, rand_rotation_matrix()), unit.nanometers)
        rotation = rotPos + comPos
        self.rotation = rotation
        return self.rotation
    def rand_rotation_matrix():
        """
        Creates a uniform random rotation matrix
        Returns
        -------
        matrix_out: 3x3 np.array
            random rotation matrix
        """
        rand_quat = md.utils.uniform_quaternion()
        matrix_out = md.utils.rotation_matrix_from_quaternion(rand_quat)
        return matrix_out
    
    def create_alchemicalSystem(self, topology, reference_system, atomsIdx=None):
        #Initialize Alchemical Simulation
        # performs alchemical corrections
        # Reporter for NCMC moves
        alchemyIntegrator = openmm.LangevinIntegrator(temperature, friction, dt)
        alchemySim = app.Simulation(prmtop.topology, system, alchemyIntegrator)
        #alchemySim.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
        # Generate Alchemical System
        factory = AbsoluteAlchemicalFactory(system, ligand_atoms=atomsIdx,
                                            annihilate_sterics=True,
                                            annihilate_electrostatics=True)
        alchemySystem = factory.createPerturbedSystem()
        return self.alchemicalSystem
    
    def setPositions(self, context, atomsIdx, newpositions):
        positions = self.getPositions(context, atomsIdx)
        for ele, idx in enumerate(atomsIdx):
            positions[idx] = newpositions[ele]
        self.positions = positions
        self.context = context.setPositions(positions)
        return self.positions